### Loading Data
We load the UofT dataset since it is the most complete dataset from last year

In [1]:
import pandas as pd
import numpy as np
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from helpers.processing import *
import glob
plotly.offline.init_notebook_mode(connected=True)

In [2]:
dfs = []
for f in glob.glob("../Data.nosync/CSVs/UofT/*.csv"): #loads every csv into the list as a pd dataframe
    df = remove_stopped(pd.read_csv(f, header=12, low_memory=False)[1:], 10, 10, 0.002) #remove ticks where car is stopped
    df = smooth_points(df.astype(float), 5) #average every 5 samples to reduce memory load (tickrate 0.002 -> 0.01)
    dfs.append(df)

### Graphing
By graphing the acceleration against the brake pressure we can evaluate the brake performance thoughout the runs. We cut off samples with speed below 50 to reduce visual clutter.

In [13]:
dfs[0].head(3) #Sanity Check - If the first three rows are correct, it follows by undergraduate induction that everything is fine.

,Time,Distance,Wheel Speed FL,Wheel Speed FR,Wheel Speed RL,Wheel Speed RR,Susp Pos FL,Susp Pos FR,Susp Pos RL,Susp Pos RR,...,VC_TORQUE_COMMAND,Battery Pack Voltage,Battery Pack Current,Battery Fan,Radiator Fan,BMS STATUS,Data Validity Status Flag,x,y,close
500,5.003,4.0,12.50,13.30,0.5,13.225,-3.485,-58.475,-2.8025,-8.930,...,0.0,291.620,-12.115,0.0,0.0,0.0,0.0,0.573427,0.871359,-1
501,5.012,4.0,12.52,13.48,0.5,13.480,-3.448,-58.990,-2.3060,-8.894,...,0.0,291.620,-12.104,0.0,0.0,0.0,0.0,0.573427,0.871359,-1
502,5.022,4.0,12.60,13.14,0.5,13.400,-3.416,-59.008,-2.0260,-8.918,...,0.0,291.612,-12.116,0.0,0.0,0.0,0.0,0.573427,0.871359,-1


We can then plot the acceleration (negative in this case), against the brake pressure, with color representing the ground speed. Use the dropdown to select which run the data is from.

In [14]:
plt_data = []
for i in range(len(dfs)):
    df = dfs[i].loc[(dfs[i]['Ground Speed'] > 50) &(dfs[i]['Brake Pres Front'] > 50)]
    plt_data += [go.Scatter(
        x=df['Brake Pres Front'], 
        y=df['Vehicle Accel Long'],
        mode='markers',
        marker=dict(color=df['Ground Speed'], colorscale='Viridis', showscale=True), 
        visible=(True if i == 0 else False))
                ]

menu = list([ dict(active=0, 
                   buttons=[ dict(label = 'Run ' + str(i+1), 
                                  method = 'update', 
                                  args = [{'visible': [False if j != i else True for j in range(len(dfs))]}, {'title': 'Run '+str(i +1) + ' Braking'}]) for i in range(len(dfs))]
                  ) ])
plt_layout = go.Layout(title='Brake Efficacy', 
                       yaxis=dict(title='Acceleration'), 
                       xaxis=dict(title='Brake Pressure'), 
                       updatemenus=menu)

fig = go.Figure(data=plt_data, layout=plt_layout)

plotly.offline.plot(fig,filename='JupyterUno')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/plotly/offline/offline.py:524: UserWarning:

Your filename `JupyterUno` didn't end with .html. Adding .html to the end of your file.



'JupyterUno.html'

### Full Pressure
You might notice that little line on the right hand side of the graph for each run. Those are, of course, the points where we apply maximal brake pressure. Let's have a closer look.

In [15]:
plt_data = []
for i in range(len(dfs)):
    df = dfs[i].loc[(dfs[i]['Brake Pres Front'] > 179)]
    df = df.groupby(df['Ground Speed'] // 25).mean() #group the dataframe into splits of 25 km
    df['decel'] = df['Vehicle Accel Long'] * -1
    plt_data += [go.Bar(x= ['25','50','75','100','125','150','175','200'], y=df['decel'], name='Run ' + str(i+1))]

plt_layout = go.Layout(title='Brake Efficacy', yaxis=dict(title='Deceleration'), xaxis=dict(title='Ground Speed'))
fig = go.Figure(data=plt_data, layout=plt_layout)
plotly.offline.plot(fig,filename='JupyterDos')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/plotly/offline/offline.py:524: UserWarning:

Your filename `JupyterDos` didn't end with .html. Adding .html to the end of your file.



'JupyterDos.html'

Again, this is only looking at total braking performance (i.e. when the pedal is fully pressed). We can render the same graph from the earlier data as well.

In [16]:
plt_data = []
for i in range(len(dfs)):
    df = dfs[i].loc[(dfs[i]['Brake Pres Front'] > 50)]
    df = df.groupby(df['Ground Speed'] // 25).mean() #group the dataframe into splits of 25 km
    df['decel'] = df['Vehicle Accel Long'] * -1
    plt_data += [go.Bar(x= ['25','50','75','100','125','150','175','200'], y=df['decel'], name='Run ' + str(i+1))]

plt_layout = go.Layout(title='Brake Efficacy', yaxis=dict(title='Deceleration'), xaxis=dict(title='Ground Speed'))
fig = go.Figure(data=plt_data, layout=plt_layout)
plotly.offline.plot(fig,filename='JupyterTres')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/plotly/offline/offline.py:524: UserWarning:

Your filename `JupyterTres` didn't end with .html. Adding .html to the end of your file.



'JupyterTres.html'

Interestingly, run 4 has noticably reduced perfomance compared to run 3 at higher speeds. Run 6 was only slightly better than the two following runs. We can of course make the same plots for acceleration too. The bar chart below plots the average acceleration at each speed interval (i.e 0-25, 25-50 etc) when the throttle is at > 50%.

In [17]:
plt_data = []
for i in range(len(dfs)):
    df = dfs[i].loc[(dfs[i]['Throttle'] > 50)]
    df = df.groupby(df['Ground Speed'] // 25).mean() #group the dataframe into splits of 25 km
    plt_data += [go.Bar(x= ['25','50','75','100','125','150','175','200'], y=df['Vehicle Accel Long'], name='Run ' + str(i+1))]

plt_layout = go.Layout(title='Acceleration at Different Speeds', yaxis=dict(title='Accel'), xaxis=dict(title='Ground Speed'))
fig = go.Figure(data=plt_data, layout=plt_layout)
plotly.offline.plot(fig,filename='JupyterQuatro')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/plotly/offline/offline.py:524: UserWarning:

Your filename `JupyterQuatro` didn't end with .html. Adding .html to the end of your file.



'JupyterQuatro.html'

Judging from the above two graphs, the best performance out of all runs was \#6. Of course, if you prefer a scatterplot, we can do that too! 

In [18]:
plt_data = []
for i in range(len(dfs)):
    df = dfs[i].loc[(dfs[i]['Ground Speed'] > 25) &(dfs[i]['Throttle'] > 50)]
    plt_data += [go.Scatter(x=df['Throttle'], y=df['Vehicle Accel Long'],mode='markers',marker=dict(color=df['Ground Speed'], colorscale='Jet', showscale=True), visible=(True if i == 0 else False))]

menu = list([ dict(active=0, buttons=[ dict(label = 'Run ' + str(i+1), method = 'update', args = [{'visible': [False if j != i else True for j in range(len(dfs))]}, {'title': 'Run '+str(i +1) + ' Acceleration'}]) for i in range(len(dfs))]
                  ) ])
plt_layout = go.Layout(title='Acceleration', yaxis=dict(title='Acceleration'), xaxis=dict(title='Throttle'), updatemenus=menu)
fig = go.Figure(data=plt_data, layout=plt_layout)
plotly.offline.plot(fig,filename='JupyterCinco')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/plotly/offline/offline.py:524: UserWarning:

Your filename `JupyterCinco` didn't end with .html. Adding .html to the end of your file.



'JupyterCinco.html'

In [19]:
#we'll need this in a minute
def closest(x, ng):
    global last#badbadbadnotgood
    if last < 0:
        dist_lst = [distance(x, k) for k in corners]
        if min(dist_lst) < ng:
            last = int(np.argmin([distance(x, k) for k in corners]))
            return last
        else:
            return  -1
    else:
        dist_lst =  [distance(x, corners[last]), distance(x, corners[(last+1) % len(corners)])]
        if min(dist_lst) < ng:
            last = (np.argmin(dist_lst) + last)%len(corners)
            return last
        else:
            return -1
#if you call this on a constant dataframe a black hole is formed   
def normal(frame):
    return (frame - frame.min()) / (frame.max() - frame.min())


In [20]:
corners = [(0.147,0.937),(0.11, 0.552), (0.264,0.531), (0.341,0.814), (0.406, 0.659), (0.589, 0.032), (0.756, 0.058),(0.588, 0.355), (0.770, 0.357), (0.902, 0.440)]
df = pd.concat([d[300:-2000] for d in dfs[1:3]], ignore_index=True)
df = smooth_points(df, 5)#too many points otherwise
#Normalize the x, y coordinates
df['x'] = normal(df['GPS Longitude'])
df['y'] = normal(df['GPS Latitude'])
last = 0
df['close'] = df[['x','y']].apply(lambda x: closest( (x[0], x[1]), 0.1), axis=1)
plt_data = [go.Scatter(x=df['x'], y=df['y'], mode='markers', marker=dict(color=df['close'], colorscale='Jet', showscale=True))]

plt_layout = go.Layout(title='Speed Heat', yaxis=dict(title='y'), xaxis=dict(title='x'))
fig = go.Figure(data=plt_data, layout=plt_layout)
plotly.offline.plot(fig,filename='JupyterSeis')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/plotly/offline/offline.py:524: UserWarning:

Your filename `JupyterSeis` didn't end with .html. Adding .html to the end of your file.



'JupyterSeis.html'

Crude, but workable. We can then average corner speed as follows. 

In [21]:
plt_data = []
for i in range(len(dfs)):
    df = dfs[i]
    df['x'] = normal(df['GPS Longitude'])
    df['y'] = normal(df['GPS Latitude'])
    last = 0
    df['close'] = df[['x','y']].apply(lambda x: closest( (x[0], x[1]), 0.1), axis=1).map(str)
    plt_data += [go.Bar(x= [str(j) for j in range(len(corners) + 1)],y=df.groupby('close')['Ground Speed'].mean(), name = 'Run' +str(i))]

plt_layout = go.Layout(title='Corner Speed', yaxis=dict(title='Speed km/h'), xaxis=dict(title='Corner'))
fig = go.Figure(data=plt_data, layout=plt_layout)
plotly.offline.plot(fig,filename='JupyterSiete')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/plotly/offline/offline.py:524: UserWarning:

Your filename `JupyterSiete` didn't end with .html. Adding .html to the end of your file.



'JupyterSiete.html'

In [22]:
plt_data = []
for i in range(len(dfs)):
    df = dfs[i]
    X = normal(df['GPS Longitude'])
    Y = normal(df['GPS Latitude'])
    Z = df[['Susp Pos FL', 'Susp Pos RR']].mean(axis=1)
    plt_data += [go.Scatter3d(x=X, y=Y, z=Z, mode='markers', marker=dict(size=5, color=df['Time'], colorscale='Viridis', showscale=True),visible=(True if i == 0 else False))]

menu = list([ dict(active=0, buttons=[ dict(label = 'Run ' + str(i+1), method = 'update', args = [{'visible': [False if j != i else True for j in range(len(dfs))]}, {'title': 'Run '+str(i +1) + ' Suspension Position'}]) for i in range(len(dfs))]
                  ) ])
plt_layout = go.Layout(title='Suspension', yaxis=dict(title='GPS Latitude'), xaxis=dict(title='GPS Longitude'), updatemenus=menu)
fig = go.Figure(data=plt_data, layout=plt_layout)
plotly.offline.plot(fig,filename='JupyterOcho')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/plotly/offline/offline.py:524: UserWarning:

Your filename `JupyterOcho` didn't end with .html. Adding .html to the end of your file.



'JupyterOcho.html'

This one is considerably harder to read, but provides some insight into the suspension performance throughout the runs. The color represents the time in seconds, with purple being the earliest to yellow the latest. The Z axis represents the average position of the Front Left and Rear Right suspension sensors. X and Y is of course the longitude and latitude. If you orbit to a top-down view, you can see the track layout. But from the sides, it is easy to see the points where the suspsension performance is relatively consistent, and others where it is more varied. I'll try to update this to look a little nicer when I have the chance.